# About this notebook

This contains example code for Assignment 3 that we talked through together on March 6 class.

In [1]:
import random
from datasets import load_dataset
import pandas as pd

In [2]:
num_files = 5535

num_files_to_sample = num_files * 0.001
num_files_to_sample

5.535

In [3]:
file_numbers = random.sample(range(0,num_files), int(num_files_to_sample))
file_numbers

[5405, 4806, 522, 4868, 4767]

In [4]:
#%%
# just a quick reminder of what the number formatting looks like
number = 35
f"{number:05d}"

'00035'

In [5]:
#%%
pattern = "data/train-{:05d}-of-05534-*"
file_patterns = [pattern.format(x) for x in file_numbers]
file_patterns

['data/train-05405-of-05534-*',
 'data/train-04806-of-05534-*',
 'data/train-00522-of-05534-*',
 'data/train-04868-of-05534-*',
 'data/train-04767-of-05534-*']

In [6]:
#%%
data_sample = load_dataset(
    "tiiuae/falcon-refinedweb",
    data_files=file_patterns, ignore_verifications=True
)
df = pd.DataFrame(data_sample["train"])

c:\Users\nsidh\anaconda3\envs\ai\lib\site-packages\datasets\load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Generating train split:   0%|          | 0/968000015 [00:00<?, ? examples/s]

In [7]:
#%%
name = '-'.join([str(x) for x in file_numbers])
df.to_parquet(f'refinedweb_{name}.parquet')

In [8]:
name

'5405-4806-522-4868-4767'

In [10]:
# To use saved file, specify the name here
name = '5405-4806-522-4868-4767'
# name = 'refinedweb_4453-1982-3699-5149-508-282-1127-5210-4203-2213-2152-1842-5375-2118-1363-3607-4652-3485-1715-1719-1214-1952'
path = f'refinedweb_{name}.parquet'
df = pd.read_parquet(path)

In [11]:
df

,content,url,timestamp,dump,segment,image_urls
0,-\n- Music\n- Music\n- Pre Order\n- New Releas...,http://www.cmdistro.de/Item/In_This_Moment__-_...,2017-03-23 04:20:12,CC-MAIN-2017-13,1490218186774.43,[]
1,Nearly $2 billion has been wiped off bitcoin’s...,http://www.cnbcafrica.com/news/international/2...,2017-03-23 04:20:09,CC-MAIN-2017-13,1490218186774.43,[[http://d16akly8855cs7.cloudfront.net/wp-cont...
2,"Indianapolis, IN Lawyers for Items Left in Bod...",http://www.cohenandmalad.com/practice-areas/me...,2017-03-23 04:22:58,CC-MAIN-2017-13,1490218186774.43,[]
3,Page 1 Page 2 Page 3 Page 4 Page 5 Page 6\nMAL...,http://www.comandosupremo.com/1942.html,2017-03-23 04:18:05,CC-MAIN-2017-13,1490218186774.43,[]
4,Summary\nAngelica Jones' grandmother leads her...,http://www.comicextra.com/comic/firestar-1986,2017-03-23 04:20:23,CC-MAIN-2017-13,1490218186774.43,[]
...,...,...,...,...,...,...
874586,Case(s): U.S. v. Archer Daniels Midland Co. an...,https://www.justice.gov/atr/case-document/fina...,2016-07-23 19:20:24,CC-MAIN-2016-30,1469257823387.9,[]
874587,"Case(s): U.S. v. Atlas Iron Processors, Inc., ...",https://www.justice.gov/atr/case-document/resp...,2016-07-23 19:39:06,CC-MAIN-2016-30,1469257823387.9,[]
874588,"15-minute drive from downtown, Al Quafil Suit...",https://www.kayak.co.uk/Tabuk-Hotels-Al-Quafil...,2016-07-23 19:16:25,CC-MAIN-2016-30,1469257823387.9,[[/rimg/himg/bb/c0/b7/booking-441584-56081935-...
874589,"The hotel is in Bivio through the Julian Pass,...",https://www.kayak.com/Maloja-Hotels.45705.hote...,2016-07-23 19:15:49,CC-MAIN-2016-30,1469257823387.9,[]


In [12]:
len(df), df.memory_usage(index=True).sum() * 1e-6

(874591, 41.980495999999995)

In [14]:
import tldextract

In [15]:
# === GET DOMAIN AND SUBDOMAIN ===
domains = []
subdomains = []
full = []
for url in df.url.values:
    extract = tldextract.extract(url)
    domains.append(extract.domain + '.' + extract.suffix)
    subdomains.append(extract.subdomain)
    full.append(extract.subdomain + '.' + extract.domain + '.' + extract.suffix)
df["domain"] = domains
df["subdomain"] = subdomains
df["full"] = full
df['totaldocuments'] = 1

In [16]:
#%%
# === COUNT WORDS === 
df['totalwords'] = [len(x.split()) for x in df['content'].tolist()]